<a href="https://colab.research.google.com/github/anubhavgupta1/Udacity/blob/main/Frameworks/Pytorch/RNNs/Char%20RNN/anna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


!mkdir -p drive
!google-drive-ocamlfuse drive

import os
os.chdir("drive/Udacity/RNN/")
!ls

sample_data
E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 146364 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.24-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0

## Setup

In [2]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

In [3]:
# open text file and read in data as `text`
with open('Anna Karenina.txt', 'r') as f:
    text = f.read()

In [4]:
print(text[:101])

Chapter 1


Happy families are all alike; every unhappy family is unhappy in its own
way.

Everything


## Tokenization : Encode the text and map each character to an integer and vice versa


In [5]:

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
print(chars)

('p', 'l', 'u', 'd', ' ', 'a', '1', '4', 't', 'O', 'b', 'm', '"', '2', 'Y', '?', ',', 's', 'B', 'S', 'D', '5', 'y', 'j', 'i', 'v', 'r', 'x', 'R', 'H', 'k', 'P', 'L', '6', '8', "'", ';', 'M', '-', ')', 'G', 'U', '%', 'X', 'N', 'T', '/', '\n', ':', '.', '!', '7', '_', 'c', 'K', 'V', 'q', 'A', '&', '@', 'C', 'g', 'W', '(', 'z', 'J', 'n', 'f', 'E', 'I', 'Z', '0', '`', 'F', '3', '9', 'Q', 'o', '$', 'h', 'w', '*', 'e')


In [6]:
int2char = dict(enumerate(chars))
print(int2char)

{0: 'p', 1: 'l', 2: 'u', 3: 'd', 4: ' ', 5: 'a', 6: '1', 7: '4', 8: 't', 9: 'O', 10: 'b', 11: 'm', 12: '"', 13: '2', 14: 'Y', 15: '?', 16: ',', 17: 's', 18: 'B', 19: 'S', 20: 'D', 21: '5', 22: 'y', 23: 'j', 24: 'i', 25: 'v', 26: 'r', 27: 'x', 28: 'R', 29: 'H', 30: 'k', 31: 'P', 32: 'L', 33: '6', 34: '8', 35: "'", 36: ';', 37: 'M', 38: '-', 39: ')', 40: 'G', 41: 'U', 42: '%', 43: 'X', 44: 'N', 45: 'T', 46: '/', 47: '\n', 48: ':', 49: '.', 50: '!', 51: '7', 52: '_', 53: 'c', 54: 'K', 55: 'V', 56: 'q', 57: 'A', 58: '&', 59: '@', 60: 'C', 61: 'g', 62: 'W', 63: '(', 64: 'z', 65: 'J', 66: 'n', 67: 'f', 68: 'E', 69: 'I', 70: 'Z', 71: '0', 72: '`', 73: 'F', 74: '3', 75: '9', 76: 'Q', 77: 'o', 78: '$', 79: 'h', 80: 'w', 81: '*', 82: 'e'}


In [7]:
char2int = {ch: ii for ii, ch in int2char.items()}
print(char2int)

{'p': 0, 'l': 1, 'u': 2, 'd': 3, ' ': 4, 'a': 5, '1': 6, '4': 7, 't': 8, 'O': 9, 'b': 10, 'm': 11, '"': 12, '2': 13, 'Y': 14, '?': 15, ',': 16, 's': 17, 'B': 18, 'S': 19, 'D': 20, '5': 21, 'y': 22, 'j': 23, 'i': 24, 'v': 25, 'r': 26, 'x': 27, 'R': 28, 'H': 29, 'k': 30, 'P': 31, 'L': 32, '6': 33, '8': 34, "'": 35, ';': 36, 'M': 37, '-': 38, ')': 39, 'G': 40, 'U': 41, '%': 42, 'X': 43, 'N': 44, 'T': 45, '/': 46, '\n': 47, ':': 48, '.': 49, '!': 50, '7': 51, '_': 52, 'c': 53, 'K': 54, 'V': 55, 'q': 56, 'A': 57, '&': 58, '@': 59, 'C': 60, 'g': 61, 'W': 62, '(': 63, 'z': 64, 'J': 65, 'n': 66, 'f': 67, 'E': 68, 'I': 69, 'Z': 70, '0': 71, '`': 72, 'F': 73, '3': 74, '9': 75, 'Q': 76, 'o': 77, '$': 78, 'h': 79, 'w': 80, '*': 81, 'e': 82}


In [8]:
# encode the text
encoded = np.array([char2int[ch] for ch in text])
print(encoded, encoded.shape)

[60 79  5 ... 17 49 47] (1985223,)


In [9]:
print(text[:101])
print(encoded[:101])

Chapter 1


Happy families are all alike; every unhappy family is unhappy in its own
way.

Everything
[60 79  5  0  8 82 26  4  6 47 47 47 29  5  0  0 22  4 67  5 11 24  1 24
 82 17  4  5 26 82  4  5  1  1  4  5  1 24 30 82 36  4 82 25 82 26 22  4
  2 66 79  5  0  0 22  4 67  5 11 24  1 22  4 24 17  4  2 66 79  5  0  0
 22  4 24 66  4 24  8 17  4 77 80 66 47 80  5 22 49 47 47 68 25 82 26 22
  8 79 24 66 61]


## Pre-processing the data

In [10]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [11]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches

In [12]:
print("total charachters = ", len(encoded))

total charachters =  1985223


In [13]:
batch_size = 8
seq_length = 50
batch_size_total = batch_size * seq_length

# total number of batches we can make
n_batches = len(encoded)//batch_size_total
print("Total number of batches is ", n_batches)

Total number of batches is  4963


In [14]:
arr = encoded
# Keep only enough characters to make full batches
arr = arr[:n_batches * batch_size_total]
print(arr.shape)

(1985200,)


In [15]:
# Reshape into batch_size rows
arr = arr.reshape((batch_size, -1))
print(arr.shape)

(8, 248150)


In [16]:
# iterate through the array, one sequence at a time
for n in range(0, arr.shape[1], seq_length):
  # The features
  x = arr[:, n:n+seq_length]
  #print(x.shape)
  # The targets, shifted by one
  
  y = np.zeros_like(x)
  #print(y.shape)
  try:
    y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
  except IndexError:
    y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
  if n==0:
    print(x.shape)
    print(y.shape)   

(8, 50)
(8, 50)


In [17]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

## Test Your Implementation

In [18]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)
print(x.shape, y.shape)

(8, 50) (8, 50)


In [19]:
# printing out the first 10 items in a sequence
print('x\n', x[:, :10])
print('\ny\n', y[:, :10])


x
 [[60 79  5  0  8 82 26  4  6 47]
 [17 77 66  4  8 79  5  8  4  5]
 [82 66  3  4 77 26  4  5  4 67]
 [17  4  8 79 82  4 53 79 24 82]
 [ 4 17  5 80  4 79 82 26  4  8]
 [53  2 17 17 24 77 66  4  5 66]
 [ 4 57 66 66  5  4 79  5  3  4]
 [ 9 10  1 77 66 17 30 22 49  4]]

y
 [[79  5  0  8 82 26  4  6 47 47]
 [77 66  4  8 79  5  8  4  5  8]
 [66  3  4 77 26  4  5  4 67 77]
 [ 4  8 79 82  4 53 79 24 82 67]
 [17  5 80  4 79 82 26  4  8 82]
 [ 2 17 17 24 77 66  4  5 66  3]
 [57 66 66  5  4 79  5  3  4 17]
 [10  1 77 66 17 30 22 49  4 12]]


In [20]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


## Defining the network with PyTorch

In [21]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(), weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [22]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [23]:
print(net.init_hidden(8)[0].shape)
print(net.init_hidden(8)[1].shape)

torch.Size([2, 8, 512])
torch.Size([2, 8, 512])


## Time to train


In [24]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),"Step: {}...".format(counter),"Loss: {:.4f}...".format(loss.item()),"Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

In [25]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [26]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2675... Val Loss: 3.2365
Epoch: 1/20... Step: 20... Loss: 3.1425... Val Loss: 3.1367
Epoch: 1/20... Step: 30... Loss: 3.1421... Val Loss: 3.1244
Epoch: 1/20... Step: 40... Loss: 3.1161... Val Loss: 3.1190
Epoch: 1/20... Step: 50... Loss: 3.1447... Val Loss: 3.1173
Epoch: 1/20... Step: 60... Loss: 3.1203... Val Loss: 3.1167
Epoch: 1/20... Step: 70... Loss: 3.1082... Val Loss: 3.1153
Epoch: 1/20... Step: 80... Loss: 3.1239... Val Loss: 3.1133
Epoch: 1/20... Step: 90... Loss: 3.1248... Val Loss: 3.1076
Epoch: 1/20... Step: 100... Loss: 3.1014... Val Loss: 3.0950
Epoch: 1/20... Step: 110... Loss: 3.0801... Val Loss: 3.0631
Epoch: 1/20... Step: 120... Loss: 2.9890... Val Loss: 2.9908
Epoch: 1/20... Step: 130... Loss: 2.9565... Val Loss: 2.9351
Epoch: 2/20... Step: 140... Loss: 2.8895... Val Loss: 2.9751
Epoch: 2/20... Step: 150... Loss: 2.8173... Val Loss: 2.7853
Epoch: 2/20... Step: 160... Loss: 2.7080... Val Loss: 2.6747
Epoch: 2/20... Step: 170... Loss:

## Checkpoint

In [27]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden, 'n_layers': net.n_layers, 'state_dict': net.state_dict(),'tokens': net.chars}

In [28]:
with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

## Making Predictions

In [29]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

## Priming and generating text

In [30]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [31]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna. The
different deat of the creature, who came to her three sides, said
simple to be forgattening the country, and as she was set on
the praces of a secret of in a compasisance of thirt that he had been in such
a son, as the side was senst that they were a great feelings,
and that he was asking all the carry of the mississ, and were this
starmed of the crubler, triuper, while the conversation was
even seeing her and settle had she standed out the same any courters,
sount from the memory, and with a smile was so as had there wished
to
see him with his secret attires and their friends and something at
a last thrie of society and stoping the close time. At happiness
were almost feeling to all themer and starter, the carriage wished and
she was not because his wife they were starting, but he cannot had
been tabered for the pass of his feet and saying, and all of his
head one hand on the point.

Seriey Ivanovitch sat down.

"You're not so much," he said, trying to see the drows of the m

## Loading a checkpoint

In [32]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)

In [33]:
net = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
net.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

## Making Predictions

In [34]:
# Sample using a loaded model
print(sample(net, 2000, top_k=5, prime="And Levin said"))

And Levin said
that their subjection was she had been trying to drop the chince to
and that had, as she could be standing to him, and all, and the peasants
had as all sevarally said that all he had nothing, and what is it through
his weeling. The same stepper satisfacted and all at the mother was
attention, to be talking of the child, and stared in the significance
of the days, he had to grieve the mother, he could not have come to
her, while he had been so much he saw that he would nut have saw that
she had been made to his head, and, as it had said sight togother that
the choode was the carriage, there.

"I don't know you that I don't let it in the middle of the stathing of it, and
who's a sort of sort of the same to think about to that,."

"It's seeing her what there's a singer, but I suppress him abroad,
so all that you must give it," she said, "then you don't know, and what's that you are
not intensily to see."

"You do, and then the sound of all, in his words whone one of the pri

In [35]:
# Sample using a loaded model
print(sample(net, 2000, top_k=5, prime="Everything was in confusion"))

Everything was in confusion in
with her that seemed to be at haste to see. And this same the
servom he sat this all thas she was saying and were sitting at the sight.

"And how could there all do it?"

"We are not so sincere, and I would shoul the meeting in the
cliss of the face."

"Why don it, and the pirter that I do not allow that they, I can't, I'll
be done, and thank of the same ten, all to see it than some
one of the carm, and as I want to say something, and I have
so such as too tell me and see, and then."

The carriage would stand to have teacher their people.
The conversation was stending that the princess was a prescust. She went
to herself, his stale that in all of a misurder trater the princess. Has
spiting at home, and he was, that still materable minute of seemen
servant he had been doibing the same tailing or on her.

"You can't get out to might to tell you that he has been served. That master,
there were now so as to be so."

"Oh, then, it's a lady of making out of my 